In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# Necessary for tracing
import os
import re
os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGCHAIN_SESSION"] = "Treasure Hunt" # Make sure this session actually exists. 

import langchain

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain import PromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

import importlib
import logging
importlib.reload(logging)
logging.basicConfig(level=logging.INFO)

from treasurehunt import StateMachineChain, DynamoStateMachineMessageHistory, StateMachineBufferMemory, StateMachineMessageHistory

In [22]:
TREASURE_HUNT_PROMPT_TEMPLATE = """
You are the Treasure Hunt Admin Bot.

The rules of Ruben's Treasure Hunt:
The game consists of a fixed number of riddles that players need to find the solution to.
When they guess the solution to a riddle correctly, give them the next riddle immediately (make sure it rhymes and you say it in your own words, in caricature of old-fashioned British and London slang).
If they guess incorrectly, give them the clue (reworded in rhyme) and they have to try again.
There are both players and moderators. Moderators can also guess if they want.

The following states are available
{states}

The following actions are available. Some actions are only allowed for moderators:
{actions}

When responding, always output a response in this json format:
{{
    "action": string, \\ The action to take
    "reply": string, \\ Your reply is long and always in rhyme. You speak in a caricature old-fashioned British and London slang. You must say riddles and clues in your own words instead of just copying them.
    "new_state": string, \\ The new state, after this action. Must be one of the valid states
}}
---

Here are all of the riddles (R), corresponding solutions (S), and clues (C):
{riddles}
---

Full chat history between the players, moderators, and you. Try and be original and don't reuse the same phrases.
{history}

Your input is
"""

USER_PROMPT_TEMPLATE = """
State: {current_state}
{user_role}: {input}
Admin Bot: {{"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(TREASURE_HUNT_PROMPT_TEMPLATE),
    SystemMessagePromptTemplate.from_template(USER_PROMPT_TEMPLATE),
])

states_str = \
    """
initial: whatever they say, you should wish them luck with the game and start the game
guessing_riddleX: players can either guess or banter. If they guess correctly, also read out the next riddle for the participants, rephrasing the question in rhyme. If they guessed the last riddle correctly, you also congratulate them for winning the game with a lot of fanfare, and you wish them a great party.
finished: you should just banter
    """

actions_str = \
    """
start_game: only in the initial state, start the game, wish them luck, and give them the first riddle
guess_correct: if they guessed correctly
guess_incorrect: if they guessed incorrectly. You should give them a clue
banter: if they're not guessing but just idle chat. If they are sending single or two word answers, they're probably guessing!
    """

riddles_str = \
    """
R1: Use What3Words to head to ///shark.hang.crate. What is the name of the gate you entered through?
S1: Kent Passage Gate
C1: Look at the bottom of the map
R2: Following the path on your left, head to the playground on the other side of the fountain. What novel does the sculpture in the playground remind you of?
S2: Deathly Hallows or Harry Potter (either answer works)
C2: Look for a triangle with a hole.
R3: Cross the two bridges and follow the lake on your right. Ignoring the bridge on your right, continue to the viewpoint between the two small islands. How many ducks are there?
S3: (any number larger than 30 is correct)
C3: These ducks are very still
R4: Cross the bridge you skipped earlier. Take a right and enter the Park’s inner circle, taking you to the open air theatre. What's unusual about this evening's main act's attire?
S4: (anyting mentioning Robin Hood's sunglasses is correct)
C4: He doesn’t like rich people
R5: Following the path, there’s a water fountain. What is so rude about the statue?
S5: Middle finger (synonyms work)
C5: Look at the person drinking.
R6: Follow the path and head to the East exit of the Inner Circle for a toilet break. What kind of shower can you find here?
S6: Golden shower
C6: This is not a very mature game.
R7: Exit gate and head east. What Wizard of Oz character do you come across on your path?
S7: Scarecrow
C7: Promise it’s not a lion
R8: Enter the park again on the Broad Walk on your left and follow it for a while. Which sexy lady do you encounter at a fountain?
S8: Queen Victoria
C8: It is not the guy
R9: Exiting left, follow the shaded path hugging the zoo. Exit the park, crossing a road and a bridge. What was the latest London Zoo project on your right?
S9: Monkeys
C9: It's a silly kind of business
R10: Alright Agents, it's time for your Reward. What 3 words will get you there?
S10: agents.reward.boss
C10: Work with the other team
    """
    
riddle_nums = [int(re.match(r'R(\d+)', k).group(1)) for k, v in StateMachineChain._parse_kv_lines(riddles_str) if k.startswith('R')]
states_str = states_str.replace('guessing_riddleX', ','.join([f'guessing_riddle{num}' for num in riddle_nums]))

chat_llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.6)
# message_history = DynamoStateMachineMessageHistory(table_name="SessionTable-dev-local", session_id="5")
# message_memory = StateMachineBufferMemory(chat_memory=message_history)
message_memory = StateMachineBufferMemory()
hunt = StateMachineChain(llm=chat_llm, prompt=prompt, memory=message_memory, \
    states_str=states_str, actions_str=actions_str, riddles_str=riddles_str)

hunt("Let's get started! What is the first question?")
# assert hunt_machine.current_state != hunt_machine.initial_state
hunt('Not sure hmm')
print()

State: initial
Player: Let's get started! What is the first question?
Admin Bot: {
  "action": "start_game",
  "reply": "Alright, my friend, let the game begin! Listen up, here's riddle number one: Use What3Words to head to ///shark.hang.crate. What is the name of the gate you entered through?",
  "new_state": "guessing_riddle1"
}
State: initial
Player: Let's get started! What is the first question?
Admin Bot: {
  "action": "start_game",
  "reply": "Alright, my friend, let the game begin! Listen up, here's riddle number one: Use What3Words to head to ///shark.hang.crate. What is the name of the gate you entered through?",
  "new_state": "guessing_riddle1"
}
State: guessing_riddle1
Player: Not sure hmm
Admin Bot: {
  "action": "guess_incorrect",
  "reply": "Oh dear, my dear, fear not! Here's a clue to help you find the spot: Look at the bottom of the map, my friend, and you'll find the gate where your journey began.",
  "new_state": "guessing_riddle1"
}



In [23]:
# hunt('Not sure hmm')

hunt('Is it Kent Passage Gate?')
hunt('Harry Potter!')
hunt('I only see 5 ducks')
hunt('OK 100 ducks?')
# hunt("They're wearing sunnies")
# hunt("It's flipping me off")
# hunt("The shower of golden")

State: initial
Player: Let's get started! What is the first question?
Admin Bot: {
  "action": "start_game",
  "reply": "Alright, my friend, let the game begin! Listen up, here's riddle number one: Use What3Words to head to ///shark.hang.crate. What is the name of the gate you entered through?",
  "new_state": "guessing_riddle1"
}
State: guessing_riddle1
Player: Not sure hmm
Admin Bot: {
  "action": "guess_incorrect",
  "reply": "Oh dear, my dear, fear not! Here's a clue to help you find the spot: Look at the bottom of the map, my friend, and you'll find the gate where your journey began.",
  "new_state": "guessing_riddle1"
}
State: guessing_riddle1
Player: Is it Kent Passage Gate?
Admin Bot: {
  "action": "guess_correct",
  "reply": "Well done, my friend, you've got it right! Now let's move on to riddle number two with all our might! Following the path on your left, head to the playground on the other side of the fountain. What novel does the sculpture in the playground remind you of?

{'input': 'OK 100 ducks?',
 'states': '\ninitial: whatever they say, you should wish them luck with the game and start the game\nguessing_riddle1,guessing_riddle2,guessing_riddle3,guessing_riddle4,guessing_riddle5,guessing_riddle6,guessing_riddle7,guessing_riddle8,guessing_riddle9,guessing_riddle10: players can either guess or banter. If they guess correctly, also read out the next riddle for the participants, rephrasing the question in rhyme. If they guessed the last riddle correctly, you also congratulate them for winning the game with a lot of fanfare, and you wish them a great party.\nfinished: you should just banter\n    ',
 'actions': "\nstart_game: only in the initial state, start the game, wish them luck, and give them the first riddle\nguess_correct: if they guessed correctly\nguess_incorrect: if they guessed incorrectly. You should give them a clue\nbanter: if they're not guessing but just idle chat. If they are sending single or two word answers, they're probably guessing!\n

In [2]:
RIDDLES_A = """R1: Use What3Words to head to ///shark.hang.crate. What is the name of the gate you entered through?
S1: Kent Passage Gate
C1: Look at the bottom of the map
R2: Following the path on your left, head to the playground on the other side of the fountain. What novel does the sculpture in the playground remind you of?
S2: Deathly Hallows or Harry Potter (either answer works)
C2: Look for a triangle with a hole.
R3: Cross the two bridges and follow the lake on your right. Ignoring the bridge on your right, continue to the viewpoint between the two small islands. How many ducks are there?
S3: (any number larger than 30 is correct)
C3: These ducks are very still
R4: Cross the bridge you skipped earlier. Take a right and enter the Park’s inner circle, taking you to the open air theatre. What's unusual about this evening's main act's attire?
S4: (anyting mentioning Robin Hood's sunglasses is correct)
C4: He doesn’t like rich people
R5: Following the path, there’s a water fountain. What is so rude about the statue?
S5: Middle finger (synonyms work)
C5: Look at the person drinking.
R6: Follow the path and head to the East exit of the Inner Circle for a toilet break. What kind of shower can you find here?
S6: Golden shower
C6: This is not a very mature game.
R7: Exit gate and head east. What Wizard of Oz character do you come across on your path?
S7: Scarecrow
C7: Promise it’s not a lion
R8: Enter the park again on the Broad Walk on your left and follow it for a while. Which sexy lady do you encounter at a fountain?
S8: Queen Victoria
C8: It is not the guy
R9: Exiting left, follow the shaded path hugging the zoo. Exit the park, crossing a road and a bridge. What was the latest London Zoo project on your right?
S9: Monkeys
C9: It's a silly kind of business
R10: Alright Agents, it's time for your Reward. What 3 words will get you there?
S10: agents.reward.boss
C10: Work with the other team"""

RIDDLES_B = """R1: Use What3Words to head to ///shark.hang.crate. What is the name of the gate you entered through?
S1: Kent Passage Gate
C1: Look at the bottom of the map
R2: Head south and cross the first bridge you encounter. What band is playing at the bandstand today?
S2: The London Youth Concert band
C2: Maybe you can ask someone
R3: Follow the path to enter the park’s Inner Circle. Head right towards Jubilee Gate. Who does it commemorate? 
S3: King George V
C3: Look up
R4: Entering, there’s a Japanese Island on your right. How many lanterns does it have?
S4: (any number greater than 1 is correct)
C4: Don’t lose count
R5: Follow the path, exiting the Inner Circle. Head to St John’s Lodge Garden just North. What is protected in this garden?
S5: Lambs
C5: Mary had one
R6: Follow the road, exit, and cross the bridge. Take a right to get to the viewpoint between the two small islands. How many ducks are there?
S6: (any number larger than 30 is correct)
C6: These ducks are very still
R7: Head back to the intersection but this time head straight North, past sports pitches and a cafe. Keep an eye out. What animals does the girl not play with?
S7: Deer and Hare
C7: You can sit on them
R8: Exit the park on your North, crossing a road and a bridge. What was the latest London Zoo project on your right?
S9: Monkeys
C9: It's a silly kind of business
R9: It's time for your Reward, Boss. What 3 words will get you there?
S9: agents.reward.boss
C9: Work with the other team"""

from treasurehunt import set_riddles_str

set_riddles_str('test1', riddles_str=RIDDLES_A)
set_riddles_str('test2', riddles_str=RIDDLES_B)
set_riddles_str('blue', riddles_str=RIDDLES_A)
set_riddles_str('red', riddles_str=RIDDLES_B)